In [ ]:
# requests用于获取网页内容
# beautifulsoup用于parse_html 
# 参考教程大全：https://blog.csdn.net/slhlde/article/details/81937838
# mechanize用于模拟浏览器输入信息来搜索或登录:
# 参考：https://pypi.org/project/mechanize/
# http://stockrt.github.io/p/emulating-a-browser-in-python-with-mechanize/
# 用time延时以免被反爬虫甄别
# selenium教程：https://www.jianshu.com/p/6c82c965c014

# 反爬虫机制

反反爬虫利器！教你怎么用代理，拨号换IP：https://zhuanlan.zhihu.com/p/26876834

In [83]:
import sys
sys.getdefaultencoding() 

'utf-8'

In [84]:
# 代理池 https://www.jianshu.com/p/b76df35aec93
from fake_useragent import UserAgent
ua = UserAgent()

In [45]:
import random
ip_pool = [
    '119.98.44.192:8118',
    '111.198.219.151:8118',
    '101.86.86.101:8118',
]
def ip_proxy():
    ip = ip_pool[random.randrange(0,3)]
    proxy_ip = 'http://'+ip
    proxies = {'http':proxy_ip}
    return proxies

print(ip_proxy())

{'http': 'http://111.198.219.151:8118'}


In [89]:
import requests
from bs4 import BeautifulSoup
import time
import random
# 参考：https://blog.csdn.net/Shenpibaipao/article/details/80217737
# https://www.jianshu.com/p/b76df35aec93
# https://blog.csdn.net/ysblogs/article/details/88530124
# https://blog.csdn.net/qq_41007998/article/details/109221626

for i in range(1,7):
    headers = {'User-Agent': ua.random} # 更换headers来应对反爬虫
    page_link = 'https://academic.oup.com/applij/search-results?q=second+language+acquisition&f_ArticleTypeDisplayName=Research+Article&fl_SiteID=5135&rg_ArticleDate=12%2f01%2f2015+TO+12%2f31%2f2020&qb=%7b%22All1%22%3a%22second+language+acquisition%22%7d&page='+str(i)
    bs = BeautifulSoup(requests.get(page_link, headers=headers).content, "lxml")
    print('Scraping the ' +str(i)+ ' page...')
    for div in bs.find_all("div", class_ = "al-citation-list"):  
        headers = {'User-Agent': ua.random}
        link = div.a.contents[0]
        bs = BeautifulSoup(requests.get(link, headers=headers).content, "lxml")
        title = bs.title.contents[0]
        date = bs.find('div', class_ = 'citation-date').contents[0]
        abstract = bs.find('p', class_='chapter-para').contents
        paper_dict = {}
        paper_dict[ 'title'] = title
        paper_dict[ 'date'] = date
        paper_dict[ 'abstract'] = abstract
        paper_dict[ 'link'] = link
        paper_list.append(paper_dict)
        time.sleep(random.random()*10)
    time.sleep(random.random()*30)

Scraping the 5 page...
Scraping the 6 page...


In [205]:
import pandas as pd
df = pd.DataFrame(paper_list)
df.drop_duplicates(subset=['title','date'],keep='first',inplace=True)
df['year'] = df['date'].str.extract('(\d{4})$', expand=True)
df.sort_values(by='date', inplace=True)
df['abstract'] = [''.join(str(abstract[i]) for i in range(len(abstract))) for abstract in df['abstract']]
df['title'] = df['title'].str.extract('^(.*) \| Applied Linguistics', expand=True)

In [209]:
df.to_csv('papers.csv')